In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from robot import RobotDescription
import utils
from sklearn.covariance import MinCovDet

In [75]:


observations_file_select = 20
observations_file_str_dict = {20: r'observation_files/obs_single_marker_2023-11-01-11-12-21_20240109-060457.p',
                              21: r'observation_files/observations_simulated_20240109_081340.p'}
df_observations = pd.read_pickle(observations_file_str_dict[observations_file_select])
# nprint(df_observations)

In [96]:
[p+str(i) for p in ['theta', 'd', 'r', 'alpha'] for i in range(len(RobotDescription.dhparams[p+'_nom']))]

['theta0',
 'theta1',
 'theta2',
 'theta3',
 'theta4',
 'theta5',
 'theta6',
 'theta7',
 'theta8',
 'd0',
 'd1',
 'd2',
 'd3',
 'd4',
 'd5',
 'd6',
 'd7',
 'd8',
 'r0',
 'r1',
 'r2',
 'r3',
 'r4',
 'r5',
 'r6',
 'r7',
 'r8',
 'alpha0',
 'alpha1',
 'alpha2',
 'alpha3',
 'alpha4',
 'alpha5',
 'alpha6',
 'alpha7',
 'alpha8']

In [92]:
def get_marker_locations(df):
    marker_locations = []
    theta_nom = RobotDescription.dhparams["theta_nom"].astype(float)
    d_nom = RobotDescription.dhparams["d_nom"].astype(float)
    r_nom = RobotDescription.dhparams["r_nom"].astype(float)
    alpha_nom = RobotDescription.dhparams["alpha_nom"].astype(float)
    for record in df.to_records():
        q = np.concatenate([record['q'], np.zeros(2)])
        _loc = RobotDescription.get_marker_location(record['mat'], q, theta_nom, d_nom, r_nom, alpha_nom)
        marker_location = RobotDescription.get_alternate_tfs(_loc)
        marker_locations.append(np.concatenate([marker_location['rvec'], marker_location['tvec']]))
    return marker_locations


def split_df_by_marker_id(df):
    return [df[df['marker_id']==marker_id].copy() for marker_id in df['marker_id'].unique()]


def reject_outliers_by_mahalanobis_dist(dataframe, threshold):
    df_filtered = pd.DataFrame()
    single_marker_dfs = split_df_by_marker_id(dataframe)
    for df in single_marker_dfs:
        locations = get_marker_locations(df)
        cov_matrix = np.cov(locations)
        rank_cov_matrix = np.linalg.matrix_rank(cov_matrix)
        if rank_cov_matrix + 1 < np.shape(locations)[1]:
            # not full rank
            df_filtered = pd.concat([df_filtered, df])
        else:
            robust_cov = MinCovDet().fit(locations)
            df.loc[:, 'mahal_dist'] = robust_cov.dist_
            df = df[df['mahal_dist'] < threshold]
            df_filtered = pd.concat([df_filtered, df])
    df_filtered.reset_index()
    return df_filtered

In [93]:
reject_outliers_by_mahalanobis_dist(df_observations, 1000)

,marker_id,mat,t,q,interp_dist,mahal_dist
0,2,"[[-0.6512959642736411, 0.10584934278182084, 0....",32.2,"[0.06164954886680629, 0.18593552125506285, 0.0...",0.0,25.252632
1,2,"[[-0.6219646738877594, 0.13994134799481425, 0....",32.4,"[0.0811436023569394, 0.2441870706374587, 0.040...",0.0,10.455645
2,2,"[[-0.586827628389369, 0.17561501094231632, 0.7...",32.6,"[0.10140595695741439, 0.3049633047415341, 0.05...",0.0,18.045490
4,2,"[[-0.5520186946789184, 0.20810937543488303, 0....",32.8,"[0.12177856290696099, 0.36612849967825944, 0.0...",0.0,21.556064
5,2,"[[-0.51156111351037, 0.2434681326243177, 0.824...",33.0,"[0.1418915589314249, 0.42641800434852684, 0.07...",0.0,35.891600
...,...,...,...,...,...,...
869,5,"[[-0.6046362436865251, -0.09845332874258306, 0...",232.8,"[0.3999682466966368, -0.9225787352686821, 0.19...",0.0,6.429848
870,5,"[[-0.6023602801838269, -0.1035581780019883, 0....",233.0,"[0.3999696963809045, -0.9426820184991263, 0.19...",0.0,9.012329
871,5,"[[-0.5988711948541605, -0.10475323287982322, 0...",233.2,"[0.3999728507742457, -0.9625589754052788, 0.19...",0.0,8.035728
872,5,"[[-0.5927247887894582, -0.10961903677990024, 0...",233.4,"[0.39997626874412706, -0.9824815770678201, 0.1...",0.0,19.584915
